In [1]:
from pyspark.sql.functions import hour,minute,second,col,avg,when
from pyspark.sql import SQLContext
import pyspark.sql.functions as sql_functions
import datetime
sqlContext = SQLContext(sc)

In [2]:
df = sqlContext.read.csv("hdfs://localhost:54310/user/hduser/user_log_data.csv",header = True, inferSchema = True)

In [3]:

df.show(2)

+--------------------+-------------------+-------------------+-------------------+-------------------+
|           user_name|          idle_time|      working_hours|         start_time|           end_time|
+--------------------+-------------------+-------------------+-------------------+-------------------+
|  sahil24c@gmail.com|2019-10-24 05:05:00|2019-10-24 05:50:00|2019-10-24 08:30:02|2019-10-24 19:25:02|
|magadum.iranna@gm...|2019-10-24 02:15:00|2019-10-24 08:39:59|2019-10-24 08:30:02|2019-10-24 19:25:01|
+--------------------+-------------------+-------------------+-------------------+-------------------+
only showing top 2 rows



In [4]:
late_hours = df.filter(sql_functions.col('start_time') > '2019-10-24 09:30:02').select('user_name','start_time')
late_hours.show(2)

+-------------------+-------------------+
|          user_name|         start_time|
+-------------------+-------------------+
| rr582619@gmail.con|2019-10-24 09:35:01|
|addyp1911@gmail.com|2019-10-24 09:35:02|
+-------------------+-------------------+
only showing top 2 rows



In [5]:
late_hours.count()

50

In [6]:
df1 = late_hours.withColumn('hours', hour(late_hours['start_time']))
df1.show(2)

+-------------------+-------------------+-----+
|          user_name|         start_time|hours|
+-------------------+-------------------+-----+
| rr582619@gmail.con|2019-10-24 09:35:01|    9|
|addyp1911@gmail.com|2019-10-24 09:35:02|    9|
+-------------------+-------------------+-----+
only showing top 2 rows



In [7]:
x = df1.select('user_name','hours')
x = x.withColumn('h_sec', x['hours'] * 3600)
x.show(2)

+-------------------+-----+-----+
|          user_name|hours|h_sec|
+-------------------+-----+-----+
| rr582619@gmail.con|    9|32400|
|addyp1911@gmail.com|    9|32400|
+-------------------+-----+-----+
only showing top 2 rows



In [8]:
df2 = late_hours.withColumn('min', minute(late_hours['start_time']))
y = df2.select('user_name','min')
y = y.withColumn('m_sec', y['min'] * 60)
y.show(2)

+-------------------+---+-----+
|          user_name|min|m_sec|
+-------------------+---+-----+
| rr582619@gmail.con| 35| 2100|
|addyp1911@gmail.com| 35| 2100|
+-------------------+---+-----+
only showing top 2 rows



In [9]:
df3 = late_hours.withColumn('sec', second(late_hours['start_time']))
z = df3.select('user_name', 'sec')
z.show(2)

+-------------------+---+
|          user_name|sec|
+-------------------+---+
| rr582619@gmail.con|  1|
|addyp1911@gmail.com|  2|
+-------------------+---+
only showing top 2 rows



In [10]:
#Joining the dataframes
df4 = x.join(y, on = ['user_name'], how = 'inner')
df4.show(2)

+-------------------+-----+-----+---+-----+
|          user_name|hours|h_sec|min|m_sec|
+-------------------+-----+-----+---+-----+
| rr582619@gmail.con|    9|32400| 35| 2100|
|addyp1911@gmail.com|    9|32400| 35| 2100|
+-------------------+-----+-----+---+-----+
only showing top 2 rows



In [11]:
df5 = df4.join(z, on = ['user_name'], how = 'inner') 
df5.show(2)

+-------------------+-----+-----+---+-----+---+
|          user_name|hours|h_sec|min|m_sec|sec|
+-------------------+-----+-----+---+-----+---+
| rr582619@gmail.con|    9|32400| 35| 2100|  1|
|addyp1911@gmail.com|    9|32400| 35| 2100|  2|
+-------------------+-----+-----+---+-----+---+
only showing top 2 rows



In [12]:
df6 = df5.withColumn('late_hrs', (df5['h_sec'] + df5['m_sec'] + df5['sec'])/3600)
df6.show(2)

+-------------------+-----+-----+---+-----+---+----------------+
|          user_name|hours|h_sec|min|m_sec|sec|        late_hrs|
+-------------------+-----+-----+---+-----+---+----------------+
| rr582619@gmail.con|    9|32400| 35| 2100|  1|9.58361111111111|
|addyp1911@gmail.com|    9|32400| 35| 2100|  2|9.58388888888889|
+-------------------+-----+-----+---+-----+---+----------------+
only showing top 2 rows



In [13]:
average_late_hours = df6.select(avg('late_hrs'))
average_late_hours.show()

+------------------+
|     avg(late_hrs)|
+------------------+
|10.233694444444442|
+------------------+

